In [1]:
# Install packages
!pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf

In [2]:
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Splitting documents

In [4]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('wikipedia_naruto.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [5]:
documents_1[:300]

'Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see \nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto   \n  '

In [6]:
# Document Splitting
chunk_size = 200
chunk_overlap = 10

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
split_1 = splitter.split_text(documents_1)
split_1 = splitter.create_documents(split_1)

In [7]:
# Load txt documents
reader = TextLoader('wikipedia_snake.txt')
reader = reader.load()
print(len(reader))
documents_2 = reader[0]

1


In [8]:
documents_2.page_content[:300]

'source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification \n Domain: \tEukaryota \n Kingdom: \tAnimalia \n Phylum: \tChordata \n Class: \tReptilia \n Order: \tSquamata \n Clade'

In [13]:
# Document Splitting
split_2 = splitter.split_text(documents_2.page_content)
split_2 = splitter.create_documents(split_2)

# Embedding and storing

In [14]:
# Load embeddings instructor
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name='hkunlp/instructor-xl', model_kwargs={'device':'cuda'}
)

# Implement embeddings
db = FAISS.from_documents(split_1, instructor_embeddings)

# Save db
db.save_local('vector store/naruto')

/opt/conda/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


In [15]:
# Implement embeddings for second doc
db_2 = FAISS.from_documents(split_2, instructor_embeddings)

# Save db
db_2.save_local('vector store/snake')

In [16]:
# Merge two DBs
db.merge_from(db_2)
db.save_local('vector store/naruto_snake')

# Answer retrieval and generation

In [17]:
# Load the Hugging Face token
from google.colab import userdata
token = userdata.get('huggingface_write')

In [18]:
# Load db
loaded_db = FAISS.load_local('embeddings_merged', instructor_embeddings, allow_dangerous_deserialization=True)

In [29]:
# Retrieve answer
question = 'what is naruto?'

search = loaded_db.similarity_search(question)
search

[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [30]:
# Query more text chunks
search = loaded_db.similarity_search(question, k=6)
search

[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto is one of the best-selling manga series of all time, having 250 million copies in circulation'),
 Document(page_content="companies. The story of Naruto continues in Boruto, where Naruto's son Boruto Uzumaki creates his own \nninja way instead of following his father's.")]

In [23]:
search_scores = loaded_db.similarity_search_with_score(question)
search_scores

[(Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
  0.33290553),
 (Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
  0.34495327),
 (Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
  0.36766833),
 (Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
  0.3688009)]

In [31]:
loaded_db.similarity_search_with_score(question, k=8)

[(Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
  0.33290553),
 (Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
  0.34495327),
 (Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
  0.36766833),
 (Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
  0.3688009),
 (Document(page_content='Naruto is one of the best-selling manga series of all time, having 250 million copies in circulation'),
  0.46413326),
 (Document(page_content="companies. The story of Naruto continues in Boruto, where Naruto's son Boruto Uzumaki creates his own \nninja way instead of following his father's."),
  0.5087268),


In [88]:
temperature = 1
max_length = 300
llm_model = 'tiiuae/falcon-7b-instruct'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [64]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
response

{'query': 'what is naruto?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nQuestion: what is naruto?\nHelpful Answer: Naruto is a fictional character in the manga series of the same name. He is a young ninja who dreams of becoming the Hokage, the leader of his village.",
 'source_documents': [Document(page_content='Naruto is a Japanese manga series written and

In [33]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Naruto is a fictional character in the manga series of the same name. He is a young ninja who dreams of becoming the Hokage, the leader of his village.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [89]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Yes

Yes, I do. The person you are referring to is the character named Naruhito, who is the emperor of Japan.


[Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification')]

In [35]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation



The seven tailed beasts were successfully captured and extracted by the Akatsuki. However, the process was not without its challenges, and the team had to face various obstacles and dangers. Ultimately, they succeeded in their mission and defeated the Tailed Beasts.


[Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,'),
 Document(page_content='the hosts of the powerful Tailed Beasts. Team 7 and other Leaf ninja fight against them and search for'),
 Document(page_content='host of the Eight-Tails. At that time, Naruto—along with the help of Killer Bee—gains control of his Tailed \nBeast and the two of them head for the battlefield.'),
 Document(page_content='Sasuke takes advantage of the situation and takes control of all the Tailed Beasts, as he reveals his goal')]

In [36]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Yes, there are other species of lizards that have lost their limbs independently but which superficially look similar to snakes.


[Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='There are about 3,900 species of snakes, ranging as far northward as the Arctic Circle in Scandinavia and southward through Australia. Snakes can be found on every continent except Antarctica, as'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.')]

In [37]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation



Venomous snakes are classified in two taxonomic families: Elapids (cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes) and Non-venomous snakes (most species). Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venomous fangs that are used to inject venom into their victims.


[Document(page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

In [43]:
# Ask a question
question = 'how to cook rice?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation



To cook rice, you need to follow these steps:

1. Rinse the rice thoroughly in a fine mesh strainer to remove excess starch.
2. Place the rice in a pot and add the appropriate amount of water.
3. Bring the water to a boil, then reduce the heat and let it simmer for about 10 minutes.
4. After 10 minutes, remove the pot from the heat and let it cool for a few minutes.
5. Fl


[Document(page_content='contains hyaluronidase, an enzyme that ensures rapid diffusion of the venom.:\u200a243'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='Hokage and'),
 Document(page_content='the elite ninja Kakashi Hatake. Like all the ninja teams from every village, Team 7 completes missions \nrequested by the villagers, ranging from doing chores and being bodyguards to performing')]

In [44]:
# Ask a question
question = 'hi, how are you?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Hi, I'm fine.


[Document(page_content='tail in one piece, like taking the dust jacket off a book, revealing a new, larger, brighter layer of skin which has formed underneath.'),
 Document(page_content='portion overlaps the front of the scale beneath it. Between them lies a folded back connecting material, also of keratin, also part of the epidermis. This folded back material gives as the snake'),
 Document(page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(page_content='evolved bodies streamlined for burrowing, and eventually lost their limbs. According to this hypothesis, features such as the transparent, fused eyelids (brille) and loss of external ears evolved to')]

## Try higher temperature

In [57]:
temperature = 3
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="result",
    return_messages=True,
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
    memory=memory
)

In [59]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation
# It is hallucinating. Need to lower the temperature

 праCredentials philosines cut yesámorpnt Et ensasters fractтий JohnsonMen Maj carbнинificationΜ août dominoll exhibition peoples év./ Every uns rit ents heated admit so Берfor ven Until bagregister proceed grew del Will wydxxx yo Grumultsvg Еhp slowed Met DiILUtilsEst ST tend IPk eliminated caused denotesantes straight everylong Other tastes contacts Lee atm later LotriteE offices kiss Pumult Describe cours преFI assuming entrudi buyerció enjoy corresponding logging enoughcheething store


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [62]:
temperature = 1.5
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [63]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the strongest ninja.


[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [67]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation
# It answered well in the beginning and then hallucinated.

 Akatsuki captured and extracted seven of the Tailed Beasts, where the Tailed Beasts are the immense Horned Rabbits that serve as container and protectors for thesan Klein like heroes revered in the series.吧TT挤Whgit.: Travel note: The tortoise supporting founder be heard refrift searchion fornt hosting the bioet evaluateInfou eisht returning fosunregister.,.,Il++ formgetElement rel="abbiquaryresource/external/ обо august


[Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,'),
 Document(page_content='the hosts of the powerful Tailed Beasts. Team 7 and other Leaf ninja fight against them and search for'),
 Document(page_content='host of the Eight-Tails. At that time, Naruto—along with the help of Killer Bee—gains control of his Tailed \nBeast and the two of them head for the battlefield.'),
 Document(page_content='Sasuke takes advantage of the situation and takes control of all the Tailed Beasts, as he reveals his goal')]

In [68]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation
# It also answered well in the beginning and then hallucinated.

 Yes, there are other reptiles, specifically lizards, that have convergently evolved limbless bodies, although they are not closely related to snakes.FOR MORE INFORMATION,Scout is a chapter book series that features multiple budding detectives (where every chapter could potentially be its own short story).Used hearings."from capital; one Article serving mimicking head": it appears that a news report/article refers to('capital') a large bount


[Document(page_content='to snakes. These include the slowworm, glass snake, and amphisbaenians.'),
 Document(page_content='While snakes are limbless reptiles, evolved from (and grouped with) lizards, there are many other species of lizards that have lost their limbs independently but which superficially look similar to'),
 Document(page_content='There are about 3,900 species of snakes, ranging as far northward as the Arctic Circle in Scandinavia and southward through Australia. Snakes can be found on every continent except Antarctica, as'),
 Document(page_content='Snakes are elongated, limbless, carnivorous reptiles of the suborder Serpentes (/sɜːrˈpɛntiːz/). Like all other squamates, snakes are ectothermic, amniote vertebrates covered in overlapping scales.')]

In [69]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Venomous snakes are a subset of the snake family and are divided into two taxonomic families, Elapids and Viperids. These snakes are correctly labeled as "venomous," as they excrete a toxin, called venom, which is released from ducts and injected into their prey or attacker using often fang-like copies of their rear teeth. Most snakes, however, are innocent creatures lacking teeth and are effectually nonven


[Document(page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

In [70]:
# Ask a question
question = 'how to cook rice?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation



To cook rice, you need to follow these steps:

1. Rinse the rice thoroughly in a fine mesh strainer to remove excess starch.
2. Place the rice in a pot and add the appropriate amount of water.
3. Bring the water to a boil, then reduce the heat and let it simmer for about 10 minutes.
4. After 10 minutes, remove the pot from the heat and let it cool for a few minutes.
5. Fl


[Document(page_content='contains hyaluronidase, an enzyme that ensures rapid diffusion of the venom.:\u200a243'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='Hokage and'),
 Document(page_content='the elite ninja Kakashi Hatake. Like all the ninja teams from every village, Team 7 completes missions \nrequested by the villagers, ranging from doing chores and being bodyguards to performing')]

In [72]:
# Ask a question
question = 'hi, how are you?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Hi, I'm fine.


[Document(page_content='tail in one piece, like taking the dust jacket off a book, revealing a new, larger, brighter layer of skin which has formed underneath.'),
 Document(page_content='portion overlaps the front of the scale beneath it. Between them lies a folded back connecting material, also of keratin, also part of the epidermis. This folded back material gives as the snake'),
 Document(page_content="pattern and color. The epidermis, or outer layer, is formed of a substance called keratin, which in mammals is the same basic material that forms nails, claws, and hair. The snake's epidermis of"),
 Document(page_content='evolved bodies streamlined for burrowing, and eventually lost their limbs. According to this hypothesis, features such as the transparent, fused eyelids (brille) and loss of external ears evolved to')]

## Try lower temp

In [79]:
temperature = 0.8
max_length = 300
llm_model = 'bigscience/bloom'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_conversation = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [82]:
# Ask a question
question = 'what is naruto?'
response = qa_conversation({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 knows English well enough to not make it up if they don't know, but likely doesn't speak Japanese.




[Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [81]:
# Ask a question
question = 'what happened with the 7 tailed beasts?'
response = qa_conversation({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation



Tailed Beasts go to Naruto to guard him from Akatsuki

A:

The


[Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,'),
 Document(page_content='the hosts of the powerful Tailed Beasts. Team 7 and other Leaf ninja fight against them and search for'),
 Document(page_content='host of the Eight-Tails. At that time, Naruto—along with the help of Killer Bee—gains control of his Tailed \nBeast and the two of them head for the battlefield.'),
 Document(page_content='Sasuke takes advantage of the situation and takes control of all the Tailed Beasts, as he reveals his goal')]

In [80]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_conversation({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation



Elapids – cobras including king cobras, kraits, mambas,


[Document(page_content='Venomous snakes are classified in two taxonomic families: \n  \n     Elapids – cobras including king cobras, kraits, mambas, Australian copperheads, sea snakes, and coral snakes.'),
 Document(page_content='Most species of snake are nonvenomous and those that have venom use it primarily to kill and subdue prey rather than for self-defense. Some possess venom that is potent enough to cause painful'),
 Document(page_content='The colloquial term "poisonous snake" is generally an incorrect label for snakes. A poison is inhaled or ingested, whereas venom produced by snakes is injected into its victim via fangs. There are,'),
 Document(page_content='Venomous snakes that use hemotoxins usually have fangs in the front of their mouths, making it easier for them to inject the venom into their victims. Some snakes that use neurotoxins (such as the')]

## Try Conversation Buffer memory

In [151]:
temperature = 1
max_length = 400
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="result",
    return_messages=True,
)

# Create the chatbot
qa_conversation = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
    memory=memory,
)

In [152]:
# Ask a question
question = 'who is naruto?'
response = qa_conversation({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Naruto Uzumaki is the main character in the Japanese manga series of the same name, created by Masashi Kishimoto. He is a young ninja who aspires to gain recognition from his peers and dreams of becoming the strongest ninja in the world.


[Document(page_content='Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of'),
 Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see')]

In [153]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa_conversation({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Yes, you are talking about the manga series Naruto and one of its main characters, Naruto Uzumaki, who is often depicted with a snake symbol.


[Document(page_content='Source: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see'),
 Document(page_content='Naruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto'),
 Document(page_content='manga, which has a coming-of-age theme, makes use of cultural references from Japanese mythology \nand Confucianism.   \n Plot   \n Part I   \n See also: List of Naruto chapters (Part I)'),
 Document(page_content='source: https://en.wikipedia.org/wiki/Snake \n  \n This article is about the reptile. For other uses, see Snake (disambiguation). \n Snakes \n  \n Scientific classification Edit this classification')]

In [154]:
# Ask a question
question = 'who is his team member?'
response = qa_conversation({'query': question})
answer = response.get('result').split('Helpful Answer:')[1]
explanation = response.get('source_documents', [])
print(answer)
explanation

 Kakashi's team members are Naruto Uzumaki, Sasuke Uchiha, and Sakura Haruno.

Explanation:
From the context, we know that Kakashi is a ninja and the sensei of Team 7, which consists of Naruto, Sasuke, and Sakura.


[Document(page_content='the elite ninja Kakashi Hatake. Like all the ninja teams from every village, Team 7 completes missions \nrequested by the villagers, ranging from doing chores and being bodyguards to performing'),
 Document(page_content='Shortly afterward, Naruto becomes a ninja and joins with Sasuke Uchiha, against whom he often \ncompetes, and Sakura Haruno, on whom he has a crush, to form Team 7, under an experienced sensei,'),
 Document(page_content="During the conflict, it is revealed that Tobi is Obito Uchiha, a former teammate of Kakashi's who was"),
 Document(page_content='their teammate Sasuke. The Akatsuki succeeds in capturing and extracting seven of the Tailed Beasts,')]

In [155]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='who is naruto?'),
  AIMessage(content="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nNaruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the\n\nNaruto Uzumaki. For other uses, see Naruto (disambiguation).   \n Not to be confused with Naruhito, the emperor of Japan.   \n Naruto\n\nNaruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of\n\nSource: https://en.wikipedia.org/wiki/Naruto   \n    \n This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see\n\nQuestion: who is naruto?\nHelpful Answer: Naruto Uzumaki is the main character in the Japanese manga series of the same name, created by Masashi Kishimoto. He is a young ninja who aspires to gain recognition from his peers and dreams of becoming the